In [1]:
!pip install --upgrade transformers simpletransformers numpy pandas scikit-learn tqdm===4.50.2 tensorflow tensorflow-gpu tensorboard
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.1.1)
Requirement already up-to-date: simpletransformers in /usr/local/lib/python3.6/dist-packages (0.51.13)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.19.5)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.5)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.0)
Requirement already up-to-date: tqdm===4.50.2 in /usr/local/lib/python3.6/dist-packages (4.50.2)
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.4.0)
     |████████████████████████████████| 394.7MB 44kB/s 
Requirement already up-to-date: tensorboard in /usr/local/lib/python3.6/dist-packages (2.4.0)
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.classification import ClassificationModel

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

1.7.1+cu101
True
1


In [4]:
# pre-trained model
pt_model = ClassificationModel('roberta', 'roberta-base', use_cuda=torch.cuda.is_available())

# community model
cm_model = ClassificationModel('bert', 'KB/bert-base-swedish-cased', use_cuda=torch.cuda.is_available())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KB/bert-base-swedi

In [20]:
data = pd.read_csv('ner_dataset.csv', encoding='latin1')
data.fillna(method='ffill', inplace=True)
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [21]:
data['Sentence #'] = data['Sentence #'].apply(lambda x: str(x).replace('Sentence: ', ''))
data

,Sentence #,Word,POS,Tag
0,1,Thousands,NNS,O
1,1,of,IN,O
2,1,demonstrators,NNS,O
3,1,have,VBP,O
4,1,marched,VBN,O
...,...,...,...,...
1048570,47959,they,PRP,O
1048571,47959,responded,VBD,O
1048572,47959,to,TO,O
1048573,47959,the,DT,O


In [22]:
data['Sentence #'] = LabelEncoder().fit_transform(data['Sentence #'])

In [23]:
data

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
...,...,...,...,...
1048570,42177,they,PRP,O
1048571,42177,responded,VBD,O
1048572,42177,to,TO,O
1048573,42177,the,DT,O


In [25]:
data.rename(columns={'Sentence #': 'sentence_id', 'Word':'words','Tag':'labels'},inplace=True)

In [26]:
data

,sentance_id,words,POS,labels
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
...,...,...,...,...
1048570,42177,they,PRP,O
1048571,42177,responded,VBD,O
1048572,42177,to,TO,O
1048573,42177,the,DT,O


In [27]:
data['labels'] = data['labels'].str.upper()

In [28]:
data

,sentance_id,words,POS,labels
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
...,...,...,...,...
1048570,42177,they,PRP,O
1048571,42177,responded,VBD,O
1048572,42177,to,TO,O
1048573,42177,the,DT,O


In [33]:
X = data[['sentence_id', 'words']]
y = data['labels']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [36]:
train_data = pd.DataFrame({'sentence_id':X_train['sentence_id'], 'words':X_train['words'], 'labels':y_train})
test_data = pd.DataFrame({'sentence_id':X_test['sentence_id'], 'words':X_test['words'], 'labels':y_test})

In [37]:
train_data

,sentence_id,words,labels
658503,22328,National,B-ORG
654525,22125,several,O
214241,47757,",",O
96064,37479,forces,O
645407,21664,in,O
...,...,...,...
589256,18822,and,O
411042,9741,The,O
186688,46340,Baghdad,B-GEO
93698,36301,to,O


In [38]:
test_data

,sentence_id,words,labels
40286,8990,.,O
774564,28229,encourage,O
761243,27550,that,O
287100,3494,when,O
810115,30032,a,O
...,...,...,...
996200,39503,Department,I-ORG
798675,29453,%,O
979008,38633,U.N.,B-ORG
882712,33698,.,O


In [41]:
from simpletransformers.ner import NERModel, NERArgs

In [44]:
label = data['labels'].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [45]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [48]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [49]:
model.train_model(train_data, eval_data=test_data, acc=accuracy_score)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1499, 0.19187234844602372)

In [50]:
result, model_outputs, preds_list = model.eval_model(test_data)

In [51]:
prediction, model_output = model.predict(["What is the new name of Bangalore"])
prediction

[[{'What': 'O'},
  {'is': 'O'},
  {'the': 'O'},
  {'new': 'O'},
  {'name': 'O'},
  {'of': 'O'},
  {'Bangalore': 'B-GEO'}]]

In [52]:
result, model_outputs, preds_list

({'eval_loss': 0.17337932342419565,
  'f1_score': 0.792340213877145,
  'precision': 0.8261019878997408,
  'recall': 0.76122969098439},
 [[[[10.49,
     -0.4785,
     -2.033,
     -1.088,
     -1.283,
     -1.0205,
     -0.4436,
     1.095,
     -1.635,
     -1.47,
     -1.45,
     -2.654,
     1.444,
     -1.692,
     -2.111,
     -1.94,
     -2.145]],
   [[8.805,
     0.0877,
     -2.428,
     -1.946,
     0.857,
     -0.67,
     1.409,
     1.569,
     -2.24,
     -1.56,
     -1.601,
     -3.207,
     2.357,
     -3.066,
     -3.08,
     -2.377,
     -2.73]],
   [[9.68,
     2.8,
     -1.268,
     -1.635,
     -0.83,
     -0.3274,
     -0.765,
     0.629,
     -1.712,
     -1.897,
     -1.864,
     -3.424,
     0.36,
     -2.46,
     -2.13,
     -3.06,
     -2.312]],
   [[10.125,
     -0.2634,
     -2.2,
     -1.555,
     -0.3347,
     -0.9473,
     0.997,
     0.359,
     -1.64,
     -1.413,
     -1.412,
     -3.213,
     1.547,
     -1.877,
     -2.781,
     -1.971,
     -2.21]],
 

In [53]:
result

{'eval_loss': 0.17337932342419565,
 'f1_score': 0.792340213877145,
 'precision': 0.8261019878997408,
 'recall': 0.76122969098439}

In [54]:
import os
import tarfile

def pack_model(model_path='',file_name=''):
    files = [files for _, _, files in os.walk(model_path)][0]
    with tarfile.open(file_name + '.tar.gz', 'w:gz') as f:
        for file in files:
            f.add(f'{model_path}/{file}')

pack_model('outputs', 'ner_model')

In [55]:
os.stat('ner_model.tar.gz')

os.stat_result(st_mode=33188, st_ino=2765520, st_dev=49, st_nlink=1, st_uid=0, st_gid=0, st_size=400110213, st_atime=1610521089, st_mtime=1610521000, st_ctime=1610521000)

In [56]:
os.stat('ner_model.tar.gz').st_size / (1024 * 1024)

381.5748338699341

In [59]:
import shutil

In [62]:
shutil.move('/content/ner_model.tar.gz', '/content/drive/My Drive/ner_model.tar.gz')

'/content/drive/My Drive/ner_model.tar.gz'